In [ ]:
from old_code.dataset import *
import tensorflow as tf
from MLP import *

2025-06-30 21:10:19.436812: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-30 21:10:19.477443: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-06-30 21:10:20.197221: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
%load_ext autoreload
%autoreload 2

In [ ]:
# Test training LinearModel on 2024-08-21 data
import pickle
import os
import numpy as np
from old_code.dataset import get_scaled_split_data
from MLP import MLP
from Model import ModelConfig, Model

date_str = "2024-08-21"
data_dir = "../Features/"
test_split = list(range(1,53))
scaler_date = "2024-08-21"

# Try to load feature list from xCC.pkl
features = [  "Energy","Capacity","Voltage-8","Energy-8","Power-8"]

# Load data
X_train, X_test, y_train, y_test, scaler = get_scaled_split_data(
    date_str, data_dir, test_split, scaler_date, features
)


Training list:  [53, 54, 55]
Testing list:  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52]


In [ ]:

# ModelConfig setup (adjust as needed)
config = ModelConfig(
    optimizer= 'adam',
    learning_rate=1e-3,
    batch_size=64,
    num_epochs=1
)

# Instantiate and train model
input_dim = 5
output_dim = 1
model = MLP(config, input_dim=input_dim, output_dim=output_dim)
model.train(X_train, y_train, scaler=scaler, verbose=True, pinn_weight=.1)
y_test = y_test[:5000]
X_test = X_test[:5000]
y_pred1 = model.predict(X_test)

# model.save("test_model/linear_fit")
# model2 = LinearModel(config)
# model2.load('./test_model/linear_fit')
# y_pred2 = model2.predict(X_test)
# assert np.max(np.abs(y_pred1-y_pred2)) <= 1e-8
# print("linear fit test passed")

# model = LinearModel(config, input_dim=input_dim, output_dim=output_dim)
# model.train(X_train, y_train, scaler=scaler, verbose=True, pinn_weight=0.1)
# y_test = y_test[:5000]
# X_test = X_test[:5000]
# y_pred1 = model.predict(X_test)
# model.save("test_model/linear_fit")
# model2 = LinearModel(config)
# model2.load('./test_model/linear_fit')
# y_pred2 = model2.predict(X_test)
# assert np.max(np.abs(y_pred1-y_pred2)) <= 1e-8
# print("linear pinn test passed")

2025-06-30 21:10:26.076857: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22288 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:1a:00.0, compute capability: 8.6
2025-06-30 21:10:26.077443: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 22289 MB memory:  -> device: 1, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:67:00.0, compute capability: 8.6
2025-06-30 21:10:26.077848: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:2 with 21535 MB memory:  -> device: 2, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:68:00.0, compute capability: 8.6
2025-06-30 21:10:27.440388: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:606] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


5193/5193 [==============================] - 33s 6ms/step - loss: 2.5424e-04 - mse: 2.5424e-04 - mape: 347.9997
Training complete. Logs saved to: logs/fit/20250630-211026
157/157 [==============================] - 0s 2ms/step
simple!
INFO:tensorflow:Assets written to: test_model/linear_fit/model/assets


INFO:tensorflow:Assets written to: test_model/linear_fit/model/assets


{'config': ModelConfig(learning_rate=0.001, batch_size=64, num_epochs=1, optimizer='adam', beta_1=0.9, beta_2=0.999, momentum=0.0), 'scaler': RobustScaler()}
157/157 [==============================] - 0s 2ms/step
linear fit test passed
expoch: 0


100%|██████████| 5193/5193 [01:03<00:00, 81.44it/s]


Training complete. Logs saved to: logs/fit/20250630-211100
157/157 [==============================] - 0s 2ms/step
simple!


INFO:tensorflow:Assets written to: test_model/linear_fit/model/assets


INFO:tensorflow:Assets written to: test_model/linear_fit/model/assets


{'config': ModelConfig(learning_rate=0.001, batch_size=64, num_epochs=1, optimizer='adam', beta_1=0.9, beta_2=0.999, momentum=0.0), 'scaler': RobustScaler()}


157/157 [==============================] - 0s 2ms/step
linear pinn test passed


In [ ]:
from Model import EnsembleModel
config = ModelConfig(
    optimizer= 'adam',
    learning_rate=1e-3,
    batch_size=64,
    num_epochs=1
)

models = [MLP(config) for _ in range(5)]
em = EnsembleModel(models)
em.train(X_train, y_train)

5193/5193 [==============================] - 32s 6ms/step - loss: 3.0095e-04 - mse: 3.0095e-04 - mape: 359.7896


In [18]:
import pandas as pd

df=pd.DataFrame({
    'A': [1, 2, 3],
    'b': [1, 1, 3],
    'B': [1, 4, 3],
    'a': [1, 9, 3],
})
df

,A,b,B,a
0,1,1,1,1
1,2,1,4,9
2,3,3,3,3


In [ ]:
from Dataset import Dataset

path = Dataset("../Feature")

,A,b,B,a
0,1,1,1,1
1,2,1,4,9


In [44]:
from sklearn.preprocessing import RobustScaler

data1 = np.random.rand(100,1)
data2 = np.random.rand(100,1)
data1c = np.array([a for a in data1])

r1 = RobustScaler()
r2 = RobustScaler()

r1.fit(data1)
r2.fit(data2)

print(np.max(np.abs(r2.transform(data1) - r2.transform(r1.transform(data1)) )))

2.022759447906666


In [ ]:
from Dataset import Dataset

ds = Dataset('../Features/2024-08-21')


<bound method Dataset.add_shifted_series of <Dataset.Dataset object at 0x7fd38e14eee0>>

In [67]:
ds.add_shifted_series()

Voltage  Current   Throughput      Energy   Capacity  \
cell_num cycle                                                          
1        2      3.301722  0.00106    11.096774   17.887097  11.096774   
         2      3.301717  0.00106    11.096774   17.903226  11.096774   
         2      3.301807  0.00106    11.096774   17.919355  11.096774   
         2      3.301815  0.00106    11.129032   17.935484  11.129032   
         2      3.301893  0.00106    11.129032   17.951613  11.129032   
...                  ...      ...          ...         ...        ...   
55       21     3.598121  0.00159  2868.000000  128.387097  74.129032   
         21     3.598300  0.00159  2868.032258  128.419355  74.161290   
         21     3.598487  0.00159  2868.032258  128.435484  74.161290   
         21     3.598670  0.00159  2868.032258  128.467742  74.161290   
         21     3.598896  0.00159  2868.064516  128.483871  74.193548   

                     Time  Overpotential       dQ_dV  dVoltage/dt  dEnergy/dt  \
cell_num cycle                                                                  
1        2      1167.1410       0.015730    0.000000     0.000112    0.016129   
         2      1168.1410       0.015725    0.000000    -0.000005    0.016129   
         2      1169.1410       0.015815  129.032258     0.000090    0.016129   
         2      1170.1410       0.015506  193.548387     0.000008    0.016129   
         2      1171.1412       0.015584  193.548387     0.000078    0.016129   
...                   ...            ...         ...          ...         ...   
55       21     3962.4263       0.193683   64.516129     0.000142    0.016129   
         21     3963.4271       0.193809  129.032258     0.000179    0.032258   
         21     3964.4273       0.193996   64.516129     0.000187    0.016129   
         21     3965.4272       0.194179   64.516129     0.000183    0.032258   
         21     3966.4272       0.194316   64.516129     0.000226    0.016129   

                ...  Capacity-2     Time-2  Overpotential-2     dQ_dV-2  \
cell_num cycle  ...                                                       
1        2      ...   11.064516  1165.1409         0.015911    0.000000   
         2      ...   11.096774  1166.1409         0.015618    0.000000   
         2      ...   11.096774  1167.1410         0.015730    0.000000   
         2      ...   11.096774  1168.1410         0.015725    0.000000   
         2      ...   11.096774  1169.1410         0.015815  129.032258   
...             ...         ...        ...              ...         ...   
55       21     ...   74.096774  3960.4258         0.193398   64.516129   
         21     ...   74.129032  3961.4260         0.193541  129.032258   
         21     ...   74.129032  3962.4263         0.193683   64.516129   
         21     ...   74.161290  3963.4271         0.193809  129.032258   
         21     ...   74.161290  3964.4273         0.193996   64.516129   

                dVoltage/dt-2  dEnergy/dt-2  dCapacity/dt-2  \
cell_num cycle                                                
1        2           0.000056      0.016129        0.000000   
         2           0.000023      0.016129        0.032258   
         2           0.000112      0.016129        0.000000   
         2          -0.000005      0.016129        0.000000   
         2           0.000090      0.016129        0.000000   
...                       ...           ...             ...   
55       21          0.000205      0.032258        0.000000   
         21          0.000186      0.032258        0.032258   
         21          0.000142      0.016129        0.000000   
         21          0.000179      0.032258        0.032258   
         21          0.000187      0.016129        0.000000   

                dOverpotential/dt-2   Power-2  shifted_NEP  
cell_num cycle                                              
1        2                 0.000056  0.003500     0.000000  
         2                -0.00029

In [68]:
ds.xCC.head()

Voltage  Current  Throughput     Energy   Capacity  \
cell_num cycle                                                        
1        2      3.301722  0.00106   11.096774  17.887097  11.096774   
         2      3.301717  0.00106   11.096774  17.903226  11.096774   
         2      3.301807  0.00106   11.096774  17.919355  11.096774   
         2      3.301815  0.00106   11.129032  17.935484  11.129032   
         2      3.301893  0.00106   11.129032  17.951613  11.129032   

                     Time  Overpotential       dQ_dV  dVoltage/dt  dEnergy/dt  \
cell_num cycle                                                                  
1        2      1167.1410       0.015730    0.000000     0.000112    0.016129   
         2      1168.1410       0.015725    0.000000    -0.000005    0.016129   
         2      1169.1410       0.015815  129.032258     0.000090    0.016129   
         2      1170.1410       0.015506  193.548387     0.000008    0.016129   
         2      1171.1412       0.015584  193.548387     0.000078    0.016129   

                ...  Capacity-2     Time-2  Overpotential-2     dQ_dV-2  \
cell_num cycle  ...                                                       
1        2      ...   11.064516  1165.1409         0.015911    0.000000   
         2      ...   11.096774  1166.1409         0.015618    0.000000   
         2      ...   11.096774  1167.1410         0.015730    0.000000   
         2      ...   11.096774  1168.1410         0.015725    0.000000   
         2      ...   11.096774  1169.1410         0.015815  129.032258   

                dVoltage/dt-2  dEnergy/dt-2  dCapacity/dt-2  \
cell_num cycle                                                
1        2           0.000056      0.016129        0.000000   
         2           0.000023      0.016129        0.032258   
         2           0.000112      0.016129        0.000000   
         2          -0.000005      0.016129        0.000000   
         2           0.000090      0.016129        0.000000   

                dOverpotential/dt-2  Power-2  shifted_NEP  
cell_num cycle                                             
1        2                 0.000056   0.0035     0.000000  
         2                -0.000293   0.0035     0.144482  
         2                 0.000112   0.0035     0.144440  
         2                -0.000005   0.0035     0.144396  
         2                 0.000090   0.0035     0.144356  

[5 rows x 66 columns]

In [69]:
ds.xCC.columns

Index(['Voltage', 'Current', 'Throughput', 'Energy', 'Capacity', 'Time',
       'Overpotential', 'dQ_dV', 'dVoltage/dt', 'dEnergy/dt', 'dCapacity/dt',
       'dOverpotential/dt', 'Power', 'Voltage-8', 'Current-8', 'Throughput-8',
       'Energy-8', 'Capacity-8', 'Time-8', 'Overpotential-8', 'dQ_dV-8',
       'dVoltage/dt-8', 'dEnergy/dt-8', 'dCapacity/dt-8',
       'dOverpotential/dt-8', 'Power-8', 'Voltage-6', 'Current-6',
       'Throughput-6', 'Energy-6', 'Capacity-6', 'Time-6', 'Overpotential-6',
       'dQ_dV-6', 'dVoltage/dt-6', 'dEnergy/dt-6', 'dCapacity/dt-6',
       'dOverpotential/dt-6', 'Power-6', 'Voltage-4', 'Current-4',
       'Throughput-4', 'Energy-4', 'Capacity-4', 'Time-4', 'Overpotential-4',
       'dQ_dV-4', 'dVoltage/dt-4', 'dEnergy/dt-4', 'dCapacity/dt-4',
       'dOverpotential/dt-4', 'Power-4', 'Voltage-2', 'Current-2',
       'Throughput-2', 'Energy-2', 'Capacity-2', 'Time-2', 'Overpotential-2',
       'dQ_dV-2', 'dVoltage/dt-2', 'dEnergy/dt-2', 'dCapacity/dt-2

In [70]:
ds.yCC.head()

cell_num  cycle
1         2        0.144482
          2        0.144440
          2        0.144396
          2        0.144356
          2        0.144332
Name: NEP, dtype: float64

In [64]:
new_col = "shifted"
df = ds.xCC
series_data = ds.yCC        
if new_col is None:
    base = series_data.name or "series"
    new_col = f"shifted_{base}"

out = df.copy()
out[new_col] = 0.0

cell_idx = df.index.get_level_values("cell_num")
cycle_idx = df.index.get_level_values("cycle")
uniq = set(zip(cell_idx, cycle_idx))
uniq

{(26, 21),
 (50, 6),
 (42, 2),
 (28, 67),
 (29, 32),
 (8, 9),
 (19, 18),
 (5, 83),
 (11, 14),
 (50, 91),
 (29, 68),
 (41, 42),
 (33, 38),
 (34, 3),
 (25, 34),
 (39, 81),
 (44, 47),
 (45, 12),
 (10, 54),
 (2, 50),
 (3, 15),
 (22, 28),
 (14, 24),
 (12, 100),
 (13, 65),
 (46, 13),
 (38, 9),
 (24, 74),
 (43, 87),
 (49, 18),
 (35, 83),
 (36, 48),
 (15, 25),
 (1, 90),
 (7, 21),
 (48, 22),
 (5, 60),
 (46, 98),
 (30, 4),
 (9, 67),
 (28, 80),
 (29, 45),
 (41, 19),
 (5, 96),
 (33, 15),
 (25, 11),
 (44, 24),
 (10, 31),
 (2, 27),
 (22, 5),
 (34, 16),
 (20, 81),
 (25, 47),
 (39, 94),
 (12, 77),
 (45, 25),
 (31, 90),
 (23, 86),
 (3, 28),
 (43, 64),
 (14, 37),
 (15, 2),
 (1, 67),
 (47, 34),
 (26, 11),
 (18, 7),
 (35, 96),
 (36, 61),
 (9, 44),
 (28, 57),
 (29, 22),
 (21, 18),
 (5, 73),
 (11, 4),
 (52, 5),
 (9, 80),
 (50, 81),
 (10, 8),
 (8, 84),
 (41, 32),
 (54, 51),
 (33, 28),
 (25, 24),
 (53, 6),
 (39, 71),
 (44, 37),
 (45, 2),
 (31, 67),
 (10, 44),
 (51, 45),
 (2, 40),
 (3, 5),
 (43, 41),
 (22, 18)